In [1]:
import os
import re
import pandas as pd
import pickle
import collections
import numpy as np
import math
import ast
from time import gmtime, strftime
import re
import time
from tqdm.auto import tqdm
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', 500)

In [5]:
glowpick_product_info_final_version = pd.read_csv("glowpick_product_info_final_version.csv")
#glowpick_product_info_final_version = glowpick_product_info_final_version.iloc[:, 1:]

In [291]:
glowpick_product_info_final_version.to_csv("glowpick_product_info_final_version.csv", index=False)

In [290]:
glowpick_product_info_final_version

,id,brand_url,brand_name,brand_code,product_url,product_code,product_name,selection,division,group,product_size,product_price,rank_within_category,rank_category,rank_within_brand,product_awards,product_awards_sector,product_awards_rank,product_description,product_type_or_color,product_stores,count_hazard_ingredients_unknown,hazard_ingredients_unknown_lst_kr,hazard_ingredients_unknown_lst_eng,hazard_ingredients_unknown_description,count_hazard_ingredients_low,hazard_ingredients_low_lst_kr,hazard_ingredients_low_lst_eng,hazard_ingredients_low_description,count_hazard_ingredients_moderate,hazard_ingredients_moderate_lst_kr,hazard_ingredients_moderate_lst_eng,hazard_ingredients_moderate_description,count_hazard_ingredients_high,hazard_ingredients_high_lst_kr,hazard_ingredients_high_lst_eng,hazard_ingredients_high_description,product_rating,product_review_date,count_product_review,user_skin_type,user_age,user_sex,user_name,product_review,check_duplicates
0,0,https://www.glowpick.com/brands/8016,가가알로에,8016,https://www.glowpick.com/products/104713,104713.0,라율 나이트 케어 너리싱 젤,스킨케어,크림,수분크림,100g,"27,000원",472위,수분크림,1위,NaN,NaN,NaN,천연 알로에베라잎즙과 10여가지 이상의 천연보습인자(NMF) 성분을 더한 수분의 힘...,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,"[4, 4, 4, 5, 5, 5, 5]","['4개월전', '2021.02.14', '2021.01.23', '2020.11....","[36, 71, 59, 97, 23, 94, 148]","['복합성', '건성', '복합성', '건성', '지성', '민감성', '복합성']","[50, 33, 24, 19, 27, 27, 32]","['여', '여', '남', '여', '여', '여', '여']","['땡또이', '소롱둥', '글로우맨이좋아', '[신유진]', '예우니95', 'ㄴ...","['발림성 좋고 촉촉함도 오래가며 많은애착이 갑니다', '저는 악건성 피부라 아주 ...",NaN
1,1,https://www.glowpick.com/brands/7314,가디스가든,7314,https://www.glowpick.com/products/96142,96142.0,유기농 드림 리페어 선 리페어 나이트 크림,스킨케어,크림,나이트크림,28g,NaN,120위,나이트크림,1위,NaN,NaN,NaN,-저녁에 세안 후 얼굴 혹은 목에 발라줍니다.-유기농 성분이 포함되어 있습니다-화학...,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,[4],['4개월전'],[36],['복합성'],[50],['여'],['땡또이'],['잠들기전 꼭바르고자고 아침에 일어나면 촉촉하고 기분 좋아요'],NaN
2,2,https://www.glowpick.com/brands/7314,가디스가든,7314,https://www.glowpick.com/products/96141,96141.0,유기농 데이 언던 선 리페어 세럼,스킨케어,에센스/세럼,진정에센스,30ml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제품에 포함되어있는 망고버터는 피부를 촉촉하게 해주고 석류 오일은 피부를 보호하고 ...,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,https://www.glowpick.com/brands/7314,가디스가든,7314,https://www.glowpick.com/products/96138,96138.0,천연 미네랄 선스크린 립 밤 라즈베리 맛,립메이크업,립케어,립밤,4g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-해에 노출되기 15분전 발라주십시오-80분정도 워터프루프가 가능합니다-오가닉 성분...,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,https://www.glowpick.com/brands/7314,가디스가든,7314,https://www.glowpick.com/products/96133,96133.0,유기농 브라이트 아이즈 퍼밍 아이 크림,스킨케어,크림,아이크림,14g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,피곤해보이는 눈을 없애기 위해 눈밑에 사용해서 잔주름을 부드럽게 하고 눈의 붓기를 빼세요,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112010,112010,https://www.glowpick.com/brands/7405,비바이바닐라,7405,https://www.glowpick.com/products/150328,150328.0,하이라이터 브러시,뷰티툴,브러시,치크브러시,1ea,"9,000원",220위,치크브러시,43위,NaN,NaN,NaN,* 본 제품은 리뉴얼된 제품입니다.섬세하고 자연스러운 광채를 연출해주는 하이라이터 ...,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,[3],['1개월전'],[1237],['민감성'],[27],['여'],['신인간주연론'],['우연한 기회로 받아 써봄3333. 내가 하이라이터는 얼굴 T존에만 폴폴 발라주므...,NaN
112011,112011,https://www.glowpick.com/brands/7405,비바이바닐라,7405,https://www.glowpick.com/products/150331,150331.0,스머지 브러쉬,뷰티툴,브러시,아이섀도우브러시,1ea,"7,000원",197위,아이섀도우브러시,35위,NaN,NaN,NaN,* 본 제품은 리뉴얼된 제품입니다.포인트 컬러를 정교하게 스머지하도록 도와주는 비건...,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,"[4, 4]","['8일전', '1개월전']","[213, 1237]","['지성', '민감성']","[37, 27]","['여', '여']","['전티', '신인간주연론']",['바닐라코랑 유리카랑 콜라보해서 바닐라코 브러쉬 전부랑 유리카 브러쉬 세척기랑 세...,NaN
112012,112012,https://www.glowp

In [6]:
glowpick_product_info_final_version.columns

Index(['id', 'brand_url', 'brand_name', 'brand_code', 'product_url',
       'product_code', 'product_name', 'selection', 'division', 'group',
       'product_size', 'product_price', 'rank_within_category',
       'rank_category', 'rank_within_brand', 'product_awards',
       'product_awards_sector', 'product_awards_rank', 'product_description',
       'product_type_or_color', 'product_stores',
       'count_hazard_ingredients_unknown', 'hazard_ingredients_unknown_lst_kr',
       'hazard_ingredients_unknown_lst_eng',
       'hazard_ingredients_unknown_description',
       'count_hazard_ingredients_low', 'hazard_ingredients_low_lst_kr',
       'hazard_ingredients_low_lst_eng', 'hazard_ingredients_low_description',
       'count_hazard_ingredients_moderate',
       'hazard_ingredients_moderate_lst_kr',
       'hazard_ingredients_moderate_lst_eng',
       'hazard_ingredients_moderate_description',
       'count_hazard_ingredients_high', 'hazard_ingredients_high_lst_kr',
       'hazard_ingr

---
## convert missing value to nan

(undeterminded price, unregistered member, empty value('') etc.)

In [254]:
check_lst = list(set(glowpick_product_info_final_version['user_age'].values))

In [255]:
len(check_lst)

52574

In [195]:
check_lst

['5개',
 '34개',
 '30개',
 '39개',
 '10개',
 '19개',
 '75개',
 '11개',
 '32개',
 '51개',
 '176개',
 '44개',
 '335개',
 '36개',
 '18개',
 '7개',
 '20개',
 '13개',
 '17개',
 '33개',
 '9개',
 '16개',
 '28개',
 '25개',
 '55개',
 '100개',
 '43개',
 '35개',
 '2개',
 '6개',
 '24개',
 '31개',
 '129개',
 '45개',
 '3개',
 '21개',
 '37개',
 '26개',
 '23개',
 '15개',
 '12개',
 '46개',
 '66개',
 '0개',
 '38개',
 '22개',
 '14개',
 '40개',
 '42개',
 '27개',
 '8개',
 '29개',
 '4개',
 '1개']

In [112]:
glowpick_product_info_final_version.loc[glowpick_product_info_final_version['product_awards_rank'] == check_lst[4]]

,id,brand_url,brand_name,brand_code,product_url,product_code,product_name,selection,division,group,product_size,product_price,rank_within_category,rank_category,rank_within_brand,product_awards,product_awards_sector,product_awards_rank,product_description,product_type_or_color,product_stores,count_hazard_ingredients_unknown,hazard_ingredients_unknown_lst_kr,hazard_ingredients_unknown_lst_eng,hazard_ingredients_unknown_description,count_hazard_ingredients_low,hazard_ingredients_low_lst_kr,hazard_ingredients_low_lst_eng,hazard_ingredients_low_description,count_hazard_ingredients_moderate,hazard_ingredients_moderate_lst_kr,hazard_ingredients_moderate_lst_eng,hazard_ingredients_moderate_description,count_hazard_ingredients_high,hazard_ingredients_high_lst_kr,hazard_ingredients_high_lst_eng,hazard_ingredients_high_description,product_rating,product_review_date,count_product_review,user_skin_type,user_age,user_sex,user_name,product_review,check_duplicates
73117,73117,https://www.glowpick.com/brands/182,유리아쥬,182,https://www.glowpick.com/products/10669,10669.0,진-8,생활용품,여성용품,여성청결제,100ml,"18,000원",10위,여성청결제,10위,"['2020상반기글로우픽어워드', '2019글로우픽컨슈머뷰티어워드', '2019상반...","['여성청결제부문', '생활용품/여성청결제부문', '바디/헤어/청결제부문', '바디...","['3위', '1위', '3위', '3위', '2위', '1위', nan]","* 본 제품은 리뉴얼 된 제품입니다. 외음부의 불편함, 불쾌한 냄새를 빠르게 씻어내...",NaN,"['롭스', '랄라블라', '올리브영', '시코르']",1개,['에델바이스꽃/잎추출물'],['LeontopodiumAlpinumFlower/LeafExtract'],[nan],11개,"['정제수', '소듐클로라이드', '소듐코코암포아세테이트', '글리세린', '코코-...","['Water;Aqua', 'SodiumChloride', 'SodiumCocoam...","['피부컨디셔닝제,용제', '감미제,점증제-수용성', '모발컨디셔닝제,계면활성제-세...",8개,"['소듐라우레스설페이트', '소듐라우레스-8설페이트', '피이지-80글리세릴코코에이...","['SodiumLaurethSulfate', 'SodiumLaureth-8Sulfa...","['계면활성제-세정제,계면활성제-유화제', '계면활성제-세정제', '계면활성제-세정...",2개,"['향료', '디엠디엠하이단토인']","['Fragrance;Perfume;Parfum', 'DMDMHydantoin']","['착향제', '살균보존제']","[4, 5, 5, 5, 4, 5, 1, 3, 4, 5, 4, 4, 5, 4, 4, ...","['25일전', '1개월전', '1개월전', '1개월전', '1개월전', '1개월전...","[88, 238, 7, 88, 64, 305, 60, 73, 192, 173, 56...","['복합성', '민감성', '건성', '지성', '건성', '복합성', '중성', ...","[27, 25, 48, 21, 29, 25, 27, 33, 28, 31, 26, 2...","['여', '여', '여', '여', '여', '여', '여', '여', '여', ...","['시옷', 'hgdsdf', '슬비1', 'dpsk08', '한울해', '니뽄구'...",['유명한 제품이라 구매했는데 역시 유명한 제품은 이유가 있는 것 같아요. 향도 좋...,NaN


In [ ]:
for i in Dfgf_fv_age.user_age:
    subr = ast.literal_eval(i)
    idx = 0
    for j in subr:
        try:
            a = int(j)
        except:
            print(i,idx, j)
            print()
            
        idx += 1

In [60]:
p = re.compile('\d+위')

for i in check_lst:
    if type(i) == float:
        continue
    else:
        match = p.findall(i)
        if match == []:
            print(i)

In [97]:
check_lst[:5]

[nan,
 "['2위','2위','3위','3위','3위','3위']",
 "['2위', '2위', '3위', '3위', '3위']",
 "['2위','3위','1위','2위']",
 "['3위', '1위', '3위', '3위', '2위', '1위', nan]"]

In [98]:
# regex check
p = re.compile('\d+위')

for i in check_lst:
    if type(i) == float:
        continue
    else:
        i = eval(i)
        for j in i:
            if type(j) == float:
                continue
            else:
                match = p.findall(j)
                if match == []:
                    print(i)

NameError: name 'nan' is not defined

In [699]:
len(check_lst)

308

In [601]:
for i in check_lst:
    try:
        if len(i) < 5:
            print(i)
    except TypeError:
        pass

니치향수
퍼프
-
보습크림
0


In [258]:
#string check
nan_in_lst = []

for idx in glowpick_product_info_final_version.index:
    try:
        if "member" in glowpick_product_info_final_version.loc[idx, 'user_age'] :
            nan_in_lst.append(idx)
    except TypeError:
        pass
print(len(nan_in_lst))

0


In [232]:
nan_in_lst

[1620,
 5344,
 8545,
 8788,
 9751,
 9846,
 17159,
 17421,
 17698,
 17902,
 24253,
 34256,
 34653,
 39864,
 44324,
 49994,
 52556,
 57367,
 57494,
 58532,
 60602,
 62664,
 63947,
 68736,
 72482,
 73327,
 77213,
 77765,
 79363,
 79618,
 87059,
 87869,
 88606,
 91792,
 97399,
 99170,
 99238,
 101364,
 103388,
 104900,
 106172,
 107410]

In [252]:
glowpick_product_info_final_version.loc[nan_in_lst[0], 'product_review']

['일단 너무 매트해서ㅠ 바르면 각질이 여기있어요!!하면서 알려주고ㅠㅠ 쓰다보면 구멍이 막히는지 잘안나와요',
 '2통 비웠는데, 트러블은 없었고, 좀 매트했고, 피부표현 뛰어난건 아니였는데,전체적으로 무난했어요.당연히 겔랑 리퀴드파데의 퀄리티는 못 따라오더라구요ㅠ그리고 쿠션은 확실히 타 브랜드 제품들이 점점 더 잘 나와서..솔직히 이 가격에 더 안 사게되더라구요!',
 '쓰다보면 구멍 막혀서 구멍 한두개에서만 나오고 색이 00호도 그리 밝지 않음... 그냥 케이스가 예쁘고 위생적으로 청결할 것 같다는 거 말곤.. 그리고 트러블 올라옴... 가격값은 못하는듯한데 색상 잘 맞고 피부에 맞으면 피부표현면에선 자연스럽게 매끈하고 예뻐서 쏘쏘..',
 '복합성 피부이며 속건조가 잇는 저에겐 촉촉 광채 파데인데요~커버도 생각보다 좋구 지속력도 괜찮아요 가격대비 돈 아깝단 생각안들어서 파데은 이것만 3년째 사용중이요!',
 '란제리 파데 사면서 함께 구매가격대비 매리트 있는 제품은 아님..얇게 발리고 새미 매트한 제품커버는 중, 중하 쯤 되는듯',
 '가격대비 진짜 쏘쏘임~아시아나 항공용 50달러짜리 쿠폰이 생겨서 비행기에서 싸게 잘 샀으니 망정이지 정가주고 사라 했으면 못 샀을듯 ㅠㅠ그냥 란제리보다 더 새미매트한 마무리감••커버는 쌓으면 쌓을수록 올라가는데엄청 얇게 발리는 애가 아니라 그만큼 화장한 티가 나긴 함색은 약간 내 기준 회끼 도는 상아빛 톤임근데 요거의 장점은...진짜 화장 안 한 것처럼 피부가 레알 편안함 ,,, (안 되겠다 굿굿으로 올려야쥐,,)진짜 이건 메포의 페이스앤바디 이후로 처음임하지만 치명적인 단점이 있었으니•••이거 쓰고 나면 울긋불긋 살짝 뭐가 올라옴 ㅠㅠ그래서 자주 쓰지는 못하고 어쩌다 한 번씩 씀게다가 어쩌다 한 번씩 쓰니 파데 나오는 구멍이 일부 막힘 현상 있음 ㅠㅠ맨 처음 사서 쓸 때만 골고루 나왔지 그 이후에는 절대 골고루 나오는 일이 없어 ㅠㅠ이건 약간 실망인데다가 불편함의 극치 ㅠㅠ아 그리고 색상이 그리 밝은 편은 아님21호 이상

In [ ]:
for idx in nan_in_lst:
    try:
        glowpick_product_info_final_version.at[idx, 'product_review'] = eval(glowpick_product_info_final_version.loc[idx, 'product_review'])
    except:
        pass
    for j in range(len(glowpick_product_info_final_version.loc[idx, 'product_review'])):
        if glowpick_product_info_final_version.loc[idx, 'product_review'][j] == '-':
            glowpick_product_info_final_version.loc[idx, 'product_review'][j] = np.nan
            print("convert to nan!")

In [1246]:
for i in glowpick_product_info_final_version_not_null.user_age:
    subr = ast.literal_eval(i)
    idx = 0
    for j in subr:
        try:
            a = int(j)
        except:
            print(i,idx, j)
            print()

        idx += 1

ValueError: malformed node or string: <_ast.Name object at 0x7fd2bc992e50>

In [510]:
for idx in tqdm(glowpick_product_info_final_version.index):
    try:
        glowpick_product_info_final_version.at[idx, 'product_rating'] = eval(glowpick_product_info_final_version.loc[idx, 'product_rating'])
    except TypeError:
        pass
    for j in range(len(glowpick_product_info_final_version.loc[idx, 'product_rating'])):
        if type(glowpick_product_info_final_version.loc[idx, 'product_rating'][j]) != int:
            print(idx)
            #glowpick_product_info_final_version.loc[idx, 'product_rating'][j] = np.nan

  0%|          | 0/112015 [00:00<?, ?it/s]

In [333]:
glowpick_product_info_final_version.loc[glowpick_product_info_final_version['product_type_or_color'] ==  "5ea(30000원)"]

,id,brand_url,brand_name,brand_code,product_url,product_code,product_name,section,division,group,product_size,product_price,rank_within_category,rank_category,rank_within_brand,product_awards,product_awards_sector,product_awards_rank,product_description,product_type_or_color,product_stores,count_hazard_ingredients_unknown,hazard_ingredients_unknown_lst_kr,hazard_ingredients_unknown_lst_eng,hazard_ingredients_unknown_description,count_hazard_ingredients_low,hazard_ingredients_low_lst_kr,hazard_ingredients_low_lst_eng,hazard_ingredients_low_description,count_hazard_ingredients_moderate,hazard_ingredients_moderate_lst_kr,hazard_ingredients_moderate_lst_eng,hazard_ingredients_moderate_description,count_hazard_ingredients_high,hazard_ingredients_high_lst_kr,hazard_ingredients_high_lst_eng,hazard_ingredients_high_description,product_rating,product_review_date,count_product_review,user_skin_type,user_age,user_sex,username,product_review,check_duplicates_reviews,product_siㄴze
94954,94954,https://www.glowpick.com/brands/6155,지나인스킨,6155,https://www.glowpick.com/products/114634,114634.0,셀프 에스테틱 실키 헤어 마스크,헤어,트리트먼트/팩,헤어마스크/팩,1ea,"6,000원",194위,헤어마스크/팩,10위,NaN,NaN,NaN,전문가의 손기로 관리 받은 것처럼 셀프 에스테틱 실키 헤어 마스크- 푸석푸석 손상 ...,5ea(30000원),['눙크'],1개,['서양송악잎추출물'],['HederaHelix(Ivy)LeafExtract'],[''],43개,"['정제수', '카프릴릭/카프릭트리글리세라이드;카프릴릭/카프릭트라이글리세라이드', ...","['Water;Aqua', 'Caprylic/CapricTriglyceride', ...","['피부컨디셔닝제,용제', '향료,수분증발차단제', '변성제,착향제,헤어컨디셔닝제,...",9개,"['디메치콘;다이메티콘', '베헨트리모늄클로라이드;베헨트라이모늄클로라이드', '세트...","['Dimethicone', 'BehentrimoniumChloride', 'Cet...","['기포방지제,피부보호제,피부컨디셔닝제(수분차단제)', '정전기방지제,헤어컨디셔닝제...",1개,['향료'],['Fragrance;Perfume;Parfum'],['착향제'],[5],['2018.12.24'],[225],['건성'],[30],['여'],['sm1122'],['이거 진짜 레알 굿이에요홈케어팩안에 들어있던 제품이라 썼는데사용하고나면 머릿결 ...,NaN,NaN


In [311]:
for idx in glowpick_product_info_final_version.loc[newline_in_rank, 'product_awards_rank'].index:
    glowpick_product_info_final_version.loc[idx, 'product_awards_rank'] = glowpick_product_info_final_version.loc[idx, 'product_awards_rank'].replace("\\n", '')
    glowpick_product_info_final_version.loc[idx, 'product_awards_rank'] = glowpick_product_info_final_version.loc[idx, 'product_awards_rank'].replace(" ", '')

In [253]:
for idx in nan_in_lst:
    try:
        glowpick_product_info_final_version.at[idx, 'product_review'] = eval(glowpick_product_info_final_version.loc[idx, 'product_review'])
    except:
        pass
    for j in range(len(glowpick_product_info_final_version.loc[idx, 'product_review'])):
        if glowpick_product_info_final_version.loc[idx, 'product_review'][j] == '-':
            glowpick_product_info_final_version.loc[idx, 'product_review'][j] = np.nan
            print("convert to nan!")

In [1017]:
#'undeterminded pice' to 'nan'
for idx in glowpick_product_info_final_version.index:
    if glowpick_product_info_final_version.loc[idx, 'product_price'] == '가격미정':
        glowpick_product_info_final_version.loc[idx, 'product_price'] = np.nan

In [ ]:
glowpick_product_info_final_version.to_csv("glowpick_product_info_final_version.csv", index=False)

---
### drop living goods that are not included in naver brand
(생활용품 중 네이버에 포함되어 있는 브랜드가 아니면 제거)

In [265]:
#글로우픽 브랜드와 네이브 브랜드 매핑 테이블
brand_key = pd.read_csv("brand_key_added.csv")
brand_key = brand_key.iloc[:, 1:]

In [266]:
glowpick_brand_w_naver = list(brand_key['glowpick_brand_name'])

In [268]:
living_goods_idx = []
for idx in tqdm(glowpick_product_info_final_version.index):
    if glowpick_product_info_final_version.loc[idx, 'selection'] == '생활용품':
        living_goods_idx.append(idx)

  0%|          | 0/112015 [00:00<?, ?it/s]

In [269]:
len(living_goods_idx) 
#selection이 '생활용품'인 product 782개

782

In [270]:
glowpick_product_info_final_version.loc[living_goods_idx, 'selection']

422       생활용품
479       생활용품
500       생활용품
582       생활용품
605       생활용품
          ... 
110660    생활용품
111842    생활용품
111845    생활용품
111896    생활용품
112004    생활용품
Name: selection, Length: 782, dtype: object

In [271]:
living_goods_idx_drop_lst = [] #drop할 생활용품 제품
living_goods_idx_no_drop_lst = [] #생활용품이지만 네이버에 브랜드가 있기 때문에 drop하지 않을 제품

for idx in living_goods_idx:
    if glowpick_product_info_final_version.loc[idx, 'brand_name'] not in glowpick_brand_w_naver:
        living_goods_idx_drop_lst.append(idx) #제거할 생활용품 product
    else:
        living_goods_idx_no_drop_lst.append(idx) #제거하지 않을 생활용품 product (생활용품 product이지만 브랜드가 네이버에 존재 (추후 처리))

In [272]:
len(living_goods_idx_drop_lst)
#네이버 뷰티 윈도우 브랜드에 포함되어 있지 않은 생활용품 product 464개

464

In [274]:
len(living_goods_idx_no_drop_lst) + len(living_goods_idx_drop_lst)

782

In [275]:
glowpick_product_info_final_version_drop_living_goods = glowpick_product_info_final_version.drop(living_goods_idx_drop_lst)

In [276]:
#reset index
glowpick_product_info_final_version_drop_living_goods = glowpick_product_info_final_version_drop_living_goods.reset_index(drop=True)

#drop id column
glowpick_product_info_final_version_drop_living_goods = glowpick_product_info_final_version_drop_living_goods.drop('id', axis='columns')

In [283]:
glowpick_product_info_final_version_drop_living_goods.to_csv("glowpick_product_info_final_version_drop_living_goods.csv", index=True)

In [284]:
glowpick_product_info_final_version_drop_living_goods = pd.read_csv("glowpick_product_info_final_version_drop_living_goods.csv")

In [285]:
glowpick_product_info_final_version_drop_living_goods = glowpick_product_info_final_version_drop_living_goods.rename({'Unnamed: 0': 'id'}, axis='columns')

In [287]:
glowpick_product_info_final_version_drop_living_goods.to_csv("glowpick_product_info_final_version_drop_living_goods.csv", index=False)

In [288]:
glowpick_product_info_final_version_drop_living_goods = pd.read_csv("glowpick_product_info_final_version_drop_living_goods.csv")

In [289]:
glowpick_product_info_final_version_drop_living_goods

,id,brand_url,brand_name,brand_code,product_url,product_code,product_name,selection,division,group,product_size,product_price,rank_within_category,rank_category,rank_within_brand,product_awards,product_awards_sector,product_awards_rank,product_description,product_type_or_color,product_stores,count_hazard_ingredients_unknown,hazard_ingredients_unknown_lst_kr,hazard_ingredients_unknown_lst_eng,hazard_ingredients_unknown_description,count_hazard_ingredients_low,hazard_ingredients_low_lst_kr,hazard_ingredients_low_lst_eng,hazard_ingredients_low_description,count_hazard_ingredients_moderate,hazard_ingredients_moderate_lst_kr,hazard_ingredients_moderate_lst_eng,hazard_ingredients_moderate_description,count_hazard_ingredients_high,hazard_ingredients_high_lst_kr,hazard_ingredients_high_lst_eng,hazard_ingredients_high_description,product_rating,product_review_date,count_product_review,user_skin_type,user_age,user_sex,user_name,product_review,check_duplicates
0,0,https://www.glowpick.com/brands/8016,가가알로에,8016,https://www.glowpick.com/products/104713,104713.0,라율 나이트 케어 너리싱 젤,스킨케어,크림,수분크림,100g,"27,000원",472위,수분크림,1위,NaN,NaN,NaN,천연 알로에베라잎즙과 10여가지 이상의 천연보습인자(NMF) 성분을 더한 수분의 힘...,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,"[4, 4, 4, 5, 5, 5, 5]","['4개월전', '2021.02.14', '2021.01.23', '2020.11....","[36, 71, 59, 97, 23, 94, 148]","['복합성', '건성', '복합성', '건성', '지성', '민감성', '복합성']","[50, 33, 24, 19, 27, 27, 32]","['여', '여', '남', '여', '여', '여', '여']","['땡또이', '소롱둥', '글로우맨이좋아', '[신유진]', '예우니95', 'ㄴ...","['발림성 좋고 촉촉함도 오래가며 많은애착이 갑니다', '저는 악건성 피부라 아주 ...",NaN
1,1,https://www.glowpick.com/brands/7314,가디스가든,7314,https://www.glowpick.com/products/96142,96142.0,유기농 드림 리페어 선 리페어 나이트 크림,스킨케어,크림,나이트크림,28g,NaN,120위,나이트크림,1위,NaN,NaN,NaN,-저녁에 세안 후 얼굴 혹은 목에 발라줍니다.-유기농 성분이 포함되어 있습니다-화학...,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,[4],['4개월전'],[36],['복합성'],[50],['여'],['땡또이'],['잠들기전 꼭바르고자고 아침에 일어나면 촉촉하고 기분 좋아요'],NaN
2,2,https://www.glowpick.com/brands/7314,가디스가든,7314,https://www.glowpick.com/products/96141,96141.0,유기농 데이 언던 선 리페어 세럼,스킨케어,에센스/세럼,진정에센스,30ml,NaN,NaN,NaN,NaN,NaN,NaN,NaN,제품에 포함되어있는 망고버터는 피부를 촉촉하게 해주고 석류 오일은 피부를 보호하고 ...,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,https://www.glowpick.com/brands/7314,가디스가든,7314,https://www.glowpick.com/products/96138,96138.0,천연 미네랄 선스크린 립 밤 라즈베리 맛,립메이크업,립케어,립밤,4g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-해에 노출되기 15분전 발라주십시오-80분정도 워터프루프가 가능합니다-오가닉 성분...,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,https://www.glowpick.com/brands/7314,가디스가든,7314,https://www.glowpick.com/products/96133,96133.0,유기농 브라이트 아이즈 퍼밍 아이 크림,스킨케어,크림,아이크림,14g,NaN,NaN,NaN,NaN,NaN,NaN,NaN,피곤해보이는 눈을 없애기 위해 눈밑에 사용해서 잔주름을 부드럽게 하고 눈의 붓기를 빼세요,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111546,111546,https://www.glowpick.com/brands/7405,비바이바닐라,7405,https://www.glowpick.com/products/150328,150328.0,하이라이터 브러시,뷰티툴,브러시,치크브러시,1ea,"9,000원",220위,치크브러시,43위,NaN,NaN,NaN,* 본 제품은 리뉴얼된 제품입니다.섬세하고 자연스러운 광채를 연출해주는 하이라이터 ...,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,[3],['1개월전'],[1237],['민감성'],[27],['여'],['신인간주연론'],['우연한 기회로 받아 써봄3333. 내가 하이라이터는 얼굴 T존에만 폴폴 발라주므...,NaN
111547,111547,https://www.glowpick.com/brands/7405,비바이바닐라,7405,https://www.glowpick.com/products/150331,150331.0,스머지 브러쉬,뷰티툴,브러시,아이섀도우브러시,1ea,"7,000원",197위,아이섀도우브러시,35위,NaN,NaN,NaN,* 본 제품은 리뉴얼된 제품입니다.포인트 컬러를 정교하게 스머지하도록 도와주는 비건...,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,0개,NaN,NaN,NaN,"[4, 4]","['8일전', '1개월전']","[213, 1237]","['지성', '민감성']","[37, 27]","['여', '여']","['전티', '신인간주연론']",['바닐라코랑 유리카랑 콜라보해서 바닐라코 브러쉬 전부랑 유리카 브러쉬 세척기랑 세...,NaN
111548,111548,https://www.glowp

In [294]:
list(glowpick_product_info_final_version_drop_living_goods.columns)

['id',
 'brand_url',
 'brand_name',
 'brand_code',
 'product_url',
 'product_code',
 'product_name',
 'selection',
 'division',
 'group',
 'product_size',
 'product_price',
 'rank_within_category',
 'rank_category',
 'rank_within_brand',
 'product_awards',
 'product_awards_sector',
 'product_awards_rank',
 'product_description',
 'product_type_or_color',
 'product_stores',
 'count_hazard_ingredients_unknown',
 'hazard_ingredients_unknown_lst_kr',
 'hazard_ingredients_unknown_lst_eng',
 'hazard_ingredients_unknown_description',
 'count_hazard_ingredients_low',
 'hazard_ingredients_low_lst_kr',
 'hazard_ingredients_low_lst_eng',
 'hazard_ingredients_low_description',
 'count_hazard_ingredients_moderate',
 'hazard_ingredients_moderate_lst_kr',
 'hazard_ingredients_moderate_lst_eng',
 'hazard_ingredients_moderate_description',
 'count_hazard_ingredients_high',
 'hazard_ingredients_high_lst_kr',
 'hazard_ingredients_high_lst_eng',
 'hazard_ingredients_high_description',
 'product_rating',
 

In [300]:
no_review_columns = ['id',
 'brand_url',
 'brand_name',
 'brand_code',
 'product_url',
 'product_code',
 'product_name',
 'selection',
 'division',
 'group',
 'product_size',
 'product_price',
 'rank_within_category',
 'rank_category',
 'rank_within_brand',
 'product_awards',
 'product_awards_sector',
 'product_awards_rank',
 'product_description',
 'product_type_or_color',
 'product_stores',
 'count_hazard_ingredients_unknown',
 'hazard_ingredients_unknown_lst_kr',
 'hazard_ingredients_unknown_lst_eng',
 'hazard_ingredients_unknown_description',
 'count_hazard_ingredients_low',
 'hazard_ingredients_low_lst_kr',
 'hazard_ingredients_low_lst_eng',
 'hazard_ingredients_low_description',
 'count_hazard_ingredients_moderate',
 'hazard_ingredients_moderate_lst_kr',
 'hazard_ingredients_moderate_lst_eng',
 'hazard_ingredients_moderate_description',
 'count_hazard_ingredients_high',
 'hazard_ingredients_high_lst_kr',
 'hazard_ingredients_high_lst_eng',
 'hazard_ingredients_high_description']

In [301]:
review_columns = ['id',
 'brand_url',
 'brand_name',
 'brand_code',
 'product_url',
 'product_code',
 'product_name', 
'product_rating',
 'product_review_date',
 'count_product_review',
 'user_skin_type',
 'user_age',
 'user_sex',
 'user_name',
 'product_review',
 'check_duplicates']

In [302]:
glowpick_product_info_final_version_drop_living_goods_w_review = glowpick_product_info_final_version_drop_living_goods[review_columns]
glowpick_product_info_final_version_drop_living_goods_d_review = glowpick_product_info_final_version_drop_living_goods[no_review_columns]

In [303]:
glowpick_product_info_final_version_drop_living_goods_w_review.to_csv("glowpick_product_info_final_version_drop_living_goods_w_review.csv", index=False)
glowpick_product_info_final_version_drop_living_goods_d_review.to_csv("glowpick_product_info_final_version_drop_living_goods_d_review.csv", index=False)

In [305]:
glowpick_product_info_final_version_drop_living_goods_w_review

,id,brand_url,brand_name,brand_code,product_url,product_code,product_name,product_rating,product_review_date,count_product_review,user_skin_type,user_age,user_sex,user_name,product_review,check_duplicates
0,0,https://www.glowpick.com/brands/8016,가가알로에,8016,https://www.glowpick.com/products/104713,104713.0,라율 나이트 케어 너리싱 젤,"[4, 4, 4, 5, 5, 5, 5]","['4개월전', '2021.02.14', '2021.01.23', '2020.11....","[36, 71, 59, 97, 23, 94, 148]","['복합성', '건성', '복합성', '건성', '지성', '민감성', '복합성']","[50, 33, 24, 19, 27, 27, 32]","['여', '여', '남', '여', '여', '여', '여']","['땡또이', '소롱둥', '글로우맨이좋아', '[신유진]', '예우니95', 'ㄴ...","['발림성 좋고 촉촉함도 오래가며 많은애착이 갑니다', '저는 악건성 피부라 아주 ...",NaN
1,1,https://www.glowpick.com/brands/7314,가디스가든,7314,https://www.glowpick.com/products/96142,96142.0,유기농 드림 리페어 선 리페어 나이트 크림,[4],['4개월전'],[36],['복합성'],[50],['여'],['땡또이'],['잠들기전 꼭바르고자고 아침에 일어나면 촉촉하고 기분 좋아요'],NaN
2,2,https://www.glowpick.com/brands/7314,가디스가든,7314,https://www.glowpick.com/products/96141,96141.0,유기농 데이 언던 선 리페어 세럼,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,https://www.glowpick.com/brands/7314,가디스가든,7314,https://www.glowpick.com/products/96138,96138.0,천연 미네랄 선스크린 립 밤 라즈베리 맛,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,https://www.glowpick.com/brands/7314,가디스가든,7314,https://www.glowpick.com/products/96133,96133.0,유기농 브라이트 아이즈 퍼밍 아이 크림,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111546,111546,https://www.glowpick.com/brands/7405,비바이바닐라,7405,https://www.glowpick.com/products/150328,150328.0,하이라이터 브러시,[3],['1개월전'],[1237],['민감성'],[27],['여'],['신인간주연론'],['우연한 기회로 받아 써봄3333. 내가 하이라이터는 얼굴 T존에만 폴폴 발라주므...,NaN
111547,111547,https://www.glowpick.com/brands/7405,비바이바닐라,7405,https://www.glowpick.com/products/150331,150331.0,스머지 브러쉬,"[4, 4]","['8일전', '1개월전']","[213, 1237]","['지성', '민감성']","[37, 27]","['여', '여']","['전티', '신인간주연론']",['바닐라코랑 유리카랑 콜라보해서 바닐라코 브러쉬 전부랑 유리카 브러쉬 세척기랑 세...,NaN
111548,111548,https://www.glowpick.com/brands/2012,세인트이브스,2012,https://www.glowpick.com/products/150804,150804.0,블랙헤드 클리어링 그린티 스크럽,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111549,111549,https://www.glowpick.com/brands/186,이니스프리,186,https://www.glowpick.com/products/150297,150297.0,청보리 고마쥬 필링 마스크,[3],['28일전'],[262],['복합성'],[21],['여'],['어쭈누나'],['🥗장점- 적당한 가격대에 적당한 제품력 원플 할 때 쟁여서 쓰기 딱인 무난템🥗단...,NaN
